In [ ]:
!pip install --upgrade --force-reinstall pymupdf
!pip install langdetect
!pip install PyPDF2

Cria estrutura inicial do corpus com informações do artigo

In [1]:
import json
import os
import requests
from bs4 import BeautifulSoup

estrutura_inicial = [];

with open('artigos_info.json', 'r', encoding='utf-8') as f:
  artigos_info = json.load(f)
try:
  for idx, artigo_info in enumerate(artigos_info):
    url_artigo = artigo_info['url']
    caminho_pdf = artigo_info['pdf']
    print(f"Extraindo {idx+1}/{len(artigos_info)}: {url_artigo}", end=" ")
    # Verifica se o PDF existe
    if not os.path.exists(caminho_pdf):
      print(f"PDF não encontrado: {caminho_pdf}")
      exit()

    # Extrai dados da página do artigo
    response = requests.get(url_artigo, timeout=10)
    soup = BeautifulSoup(response.content, 'html.parser')

    # Título
    titulo_tag = soup.find('h1', class_='page_title')
    titulo = titulo_tag.text.strip() if titulo_tag else "Título não encontrado"

    # Autores
    autores_tag = soup.find('ul', class_='item authors')
    autores = []

    if autores_tag:
      for li in autores_tag.find_all('li'):
        nome_tag = li.find('span', class_='name')
        afiliacao_tag = li.find('span', class_='affiliation')
        orcid_tag = li.find('span', class_='orcid')

        nome = nome_tag.text.strip() if nome_tag else ""
        afiliacao = afiliacao_tag.text.strip() if afiliacao_tag else ""
        orcid = orcid_tag.find('a')['href'] if orcid_tag and orcid_tag.find('a') else ''

        autores.append({
            'nome': nome,
            'afiliacao': afiliacao,
            'orcid': orcid
        })
          
    # Resumo (abstract)
    resumo_div = soup.find('div', class_='item abstract')
    if resumo_div:
        # Extrai todo o texto da div, ignora o "Resumo" e <br>
        texto_completo = resumo_div.get_text(separator=" ", strip=True)
        resumo = texto_completo.replace("Resumo", "", 1).strip()
    else:
        resumo = ""

    data_published_div = soup.find('div', class_='item published')
    if data_published_div:
        value_div = data_published_div.find('div', class_='value')
        if value_div:
            data_publicacao = value_div.text.strip()

    # Palavras-chave
    keywords = artigo_info["keywords"]

    estrutura = {
      "titulo": titulo,
      "informacoes_url": url_artigo,
      "idioma": "Inglês",
      "storage_key": caminho_pdf,
      "autores": autores,
      "data_publicacao": data_publicacao,
      "resumo": resumo,
      "keywords": keywords,
      "referencias": [],
      "artigo_completo_PT": "",
      "artigo_completo_EN": "",
      "artigo_tokenizado": [],
      "pos_tagger": [],
      "lema": [],
      "dep": []
    }
    estrutura_inicial.append(estrutura)
    print("✅ Extraído.")

  with open("corpus_bracis.json", 'w', encoding='utf-8') as f:
    json.dump(estrutura_inicial, f, ensure_ascii=False, indent=2)
except Exception as e:
  print(f"Erro ao processar artigo {url_artigo}: {e}")

Extraindo 1/26: https://sol.sbc.org.br/index.php/bracis/article/view/33550 ✅ Extraído.
Extraindo 2/26: https://sol.sbc.org.br/index.php/bracis/article/view/33647 ✅ Extraído.
Extraindo 3/26: https://sol.sbc.org.br/index.php/bracis/article/view/33556 ✅ Extraído.
Extraindo 4/26: https://sol.sbc.org.br/index.php/bracis/article/view/33560 ✅ Extraído.
Extraindo 5/26: https://sol.sbc.org.br/index.php/bracis/article/view/33562 ✅ Extraído.
Extraindo 6/26: https://sol.sbc.org.br/index.php/bracis/article/view/33648 ✅ Extraído.
Extraindo 7/26: https://sol.sbc.org.br/index.php/bracis/article/view/33649 ✅ Extraído.
Extraindo 8/26: https://sol.sbc.org.br/index.php/bracis/article/view/33654 ✅ Extraído.
Extraindo 9/26: https://sol.sbc.org.br/index.php/bracis/article/view/33576 ✅ Extraído.
Extraindo 10/26: https://sol.sbc.org.br/index.php/bracis/article/view/33579 ✅ Extraído.
Extraindo 11/26: https://sol.sbc.org.br/index.php/bracis/article/view/33629 ✅ Extraído.
Extraindo 12/26: https://sol.sbc.org.br/i

Preenche campo "referencias" no corpus

In [2]:
import json
import requests
from bs4 import BeautifulSoup
import sys


try:
    with open('corpus_bracis.json', 'r', encoding='utf-8') as f:
        corpus = json.load(f)
except Exception as e:
    print(f"Erro ao abrir o arquivo JSON: {e}")
    sys.exit
    
# Abrir os dados
with open("artigos_info.json", "r", encoding="utf-8") as f:
    artigos_info = json.load(f)

for idx, artigo in enumerate(artigos_info):
    url = artigo["springer_url"]
    try:
        print(f"🔍 Acessando {url}")
        response = requests.get(url)
        response.raise_for_status()

        soup = BeautifulSoup(response.text, "html.parser")

        # Tentar localizar seção de referências
        referencias = []
        
        paragrafos = soup.find_all("p", class_="c-article-references__text")

        for p in paragrafos:
            texto = p.get_text(strip=True)
            if texto:
                referencias.append(texto)

        corpus[idx]["referencias"] = referencias
        print(f"📚 {len(referencias)} referência(s) coletada(s).")

    except Exception as e:
        print(f"❌ Erro ao processar {url}: {e}")

# Salvar JSON atualizado
with open('corpus_bracis.json', "w", encoding="utf-8") as f:
    json.dump(corpus, f, ensure_ascii=False, indent=2)

print("✅ Finalizado.")


🔍 Acessando https://sol.sbc.org.br/index.php/bracis/article/view/33550/33342
📚 24 referência(s) coletada(s).
🔍 Acessando https://sol.sbc.org.br/index.php/bracis/article/view/33647/33439
📚 44 referência(s) coletada(s).
🔍 Acessando https://sol.sbc.org.br/index.php/bracis/article/view/33556/33348
📚 34 referência(s) coletada(s).
🔍 Acessando https://sol.sbc.org.br/index.php/bracis/article/view/33560/33352
📚 38 referência(s) coletada(s).
🔍 Acessando https://sol.sbc.org.br/index.php/bracis/article/view/33562/33354
📚 25 referência(s) coletada(s).
🔍 Acessando https://sol.sbc.org.br/index.php/bracis/article/view/33648/33440
📚 27 referência(s) coletada(s).
🔍 Acessando https://sol.sbc.org.br/index.php/bracis/article/view/33649/33441
📚 30 referência(s) coletada(s).
🔍 Acessando https://sol.sbc.org.br/index.php/bracis/article/view/33654/33446
📚 31 referência(s) coletada(s).
🔍 Acessando https://sol.sbc.org.br/index.php/bracis/article/view/33576/33368
📚 24 referência(s) coletada(s).
🔍 Acessando https:/

Insere artigos no corpus (artigo_completo_PT e artigo_completo_EN)

In [3]:
import fitz  # PyMuPDF
import json
import sys

# Caminho do arquivo
try:
    with open('corpus_bracis.json', 'r', encoding='utf-8') as f:
        corpus = json.load(f)
except Exception as e:
    print(f"Erro ao abrir o arquivo JSON: {e}")
    sys.exit()

try:
  for idx, artigo in enumerate(artigos_info):
        caminho_pdf = artigo['pdf']
        print(f"Processando: {caminho_pdf}")
      
        caminho_artigo_traduzido = 'artigos_traduzidos_pt/'+ os.path.basename(caminho_pdf).replace('.pdf', '_traduzido.txt')
        try:
            with fitz.open(caminho_pdf) as doc:
                artigo_completo_EN = ""
                for pagina in doc:
                    artigo_completo_EN += pagina.get_text()

                    # Remove referencias
            partes = artigo_completo_EN.split("References")
            if len(partes) > 1:
              artigo_completo_EN = "References".join(partes[:-1]);
        
            partes = artigo_completo_EN.split("Bibliography")
            if len(partes) > 1:
              artigo_completo_EN = "Bibliography".join(partes[:-1]);
            artigo_completo_EN = artigo_completo_EN.strip().replace("\n", " ")

            with open(caminho_artigo_traduzido, "r", encoding="utf-8") as f:
                artigo_completo_PT = f.read()
            artigo_completo_PT = artigo_completo_PT.strip().replace("\n", " ")
        except Exception as e:
            print(f"Erro ao extrair PDF {caminho_pdf}: {e}")
            sys.exit()
        corpus[idx]["artigo_completo_PT"] = artigo_completo_PT
        corpus[idx]["artigo_completo_EN"] = artigo_completo_EN
      
  with open("corpus_bracis.json", 'w', encoding='utf-8') as f:
        json.dump(corpus, f, ensure_ascii=False, indent=2)

  print("Todos os artigos foram incluidos.")

except Exception as e:
  print(f"Erro ao processar: {e}")

Processando: artigos_en/A Large Dataset of Spontaneous Speech with the Accent Spoken in São Paulo for Automatic Speech Recognition Evaluation.pdf
Processando: artigos_en/A Transformer-Based Tabular Approach to Detect Toxic Comments.pdf
Processando: artigos_en/An Ensemble of LLMs Finetuned with LoRA for NER in Portuguese Legal Documents.pdf
Processando: artigos_en/Aroeira A Curated Corpus for the Portuguese Language with a Large Number of Tokens.pdf
Processando: artigos_en/Assessing European and Brazilian Portuguese LLMs for NER in Specialised Domains.pdf
Processando: artigos_en/Automatic Text Simplification for the Legal Domain in Brazilian Portuguese.pdf
Processando: artigos_en/Developing Resource-Efficient Clinical LLMs for Brazilian Portuguese.pdf
Processando: artigos_en/Diversity in Data for Speech Processing in Brazilian Portuguese.pdf
Processando: artigos_en/ERASMO Leveraging Large Language Models for Enhanced Clustering Segmentation.pdf
Processando: artigos_en/Evaluating Large L